![Neptune + Dalex](https://neptune.ai/wp-content/uploads/2023/09/dalex.svg)

# Neptune + Dalex

<a target="_blank" href="https://colab.research.google.com/github/neptune-ai/examples/blob/main/integrations-and-supported-tools/dalex/notebooks/Neptune_Dalex.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/>
</a><a target="_blank" href="https://github.com/neptune-ai/examples/blob/main/integrations-and-supported-tools/dalex/notebooks/Neptune_Dalex.ipynb">
  <img alt="Open in GitHub" src="https://img.shields.io/badge/Open_in_GitHub-blue?logo=github&labelColor=black">
</a><a target="_blank" href="https://app.neptune.ai/o/common/org/dalex-support/runs/details?viewId=standard-view&detailsTab=dashboard&dashboardId=993ea4c1-c528-4d6d-86ba-1a7a3bd65e7e&shortId=DLX-2&type=run"> 
  <img alt="Explore in Neptune" src="https://neptune.ai/wp-content/uploads/2024/01/neptune-badge.svg">
</a><a target="_blank" href="https://docs.neptune.ai/integrations/dalex/">
  <img alt="View tutorial in docs" src="https://neptune.ai/wp-content/uploads/2024/01/docs-badge-2.svg">
</a>

## Introduction

[Dalex](https://dalex.drwhy.ai/) is an open-source tool to explore and explain model behavior to understand how complex models are working.  
This guide will show you how to:

* Upload pickled dalex explainer object to Neptune
* Upload dalex's interactive reports to Neptune.

This guide is adapted from the dalex documentation [here](https://dalex.drwhy.ai/python-dalex-titanic.html).

## Before you start

This notebook example lets you try out Neptune as an anonymous user, with zero setup.

If you want to see the example logged to your own workspace instead:

  1. Create a Neptune account. [Register &rarr;](https://neptune.ai/register)
  1. Create a Neptune project that you will use for tracking metadata. For instructions, see [Creating a project](https://docs.neptune.ai/setup/creating_project) in the Neptune docs.

## Install Neptune and dependencies

In [ ]:
%pip install -U dalex neptune pandas scikit-learn nbformat
%pip install -U --user scikit-learn

## Import libraries

In [ ]:
import dalex as dx

import pandas as pd

from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

import warnings

warnings.filterwarnings("ignore")

## Load data

In [ ]:
data = dx.datasets.load_titanic()

X = data.drop(columns="survived")
y = data.survived

data.head(10)

## Create a pipeline model

In [ ]:
numerical_features = ["age", "fare", "sibsp", "parch"]
numerical_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())]
)

categorical_features = ["gender", "class", "embarked"]
categorical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="constant", fill_value="missing")),
        ("onehot", OneHotEncoder(handle_unknown="ignore")),
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numerical_transformer, numerical_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

classifier = MLPClassifier(hidden_layer_sizes=(150, 100, 50), max_iter=500, random_state=0)

clf = Pipeline(steps=[("preprocessor", preprocessor), ("classifier", classifier)])

## Fit the model

In [ ]:
clf.fit(X, y)

### (Neptune) Start a run

To create a new run for tracking the metadata, you tell Neptune who you are (`api_token`) and where to send the data (`project`).

You can use the default code cell below to create an anonymous run in the public project [common/dalex-support](https://app.neptune.ai/o/common/org/dalex-support). **Note**: Public projects are cleaned regularly, so anonymous runs are only stored temporarily.

### Log to your own project instead

Replace the code below with the following:

```python
import neptune
from getpass import getpass

run = neptune.init_run(
    project="workspace-name/project-name",  # replace with your own (see instructions below)
    api_token=getpass("Enter your Neptune API token: "),
    tags=["reports"],  # (optional) replace with your own
)
```

To find your API token and full project name:

1. [Log in to Neptune](https://app.neptune.ai/).
1. In the bottom-left corner, expand your user menu and select **Get your API token**.
1. The workspace name is displayed in the top-left corner of the app.

    To copy the project path, in the top-right corner, open the settings menu and select **Properties**.

For more help, see [Setting Neptune credentials](https://docs.neptune.ai/setup/setting_credentials) in the Neptune docs.

In [ ]:
import neptune

run = neptune.init_run(
    api_token=neptune.ANONYMOUS_API_TOKEN,
    project="common/dalex-support",
    tags=["dalex reports"],  # (optional) replace with your own
)

**To open the run in the Neptune web app, click the link that appeared in the cell output.**

We'll use the `run` object we just created to log metadata. You'll see the metadata appear in the app.

## Create an explainer for the model¶

In [ ]:
exp = dx.Explainer(clf, X, y)

### (Neptune) Upload explainer object to Neptune
You can use dalex's [`dumps()`](https://dalex.drwhy.ai/python/api/#dalex.Explainer.dumps) method to get a pickled representation of the explainer, and then upload it to Neptune using Neptune's [`from_content()`](https://docs.neptune.ai/api/field_types/#from_content) method.

In [ ]:
from neptune.types import File

run["pickled_explainer"].upload(File.from_content(exp.dumps()))

## Model-level explanations

### model_performance

This function calculates various Model Performance measures:

- __Classification:__ F1, accuracy, recall, precision and AUC
- __Regression:__ mean squared error, R squared, median absolute deviation


In [ ]:
mp = exp.model_performance().plot(geom="roc", show=False)
mp

#### (Neptune) Upload ROC plot to Neptune
These plots can be uploaded to Neptune by setting `show=False`.  
To distinguish between the plot types, you can use namespaces. For example, "model/performace/roc", "model/performance/ecdf", etc. 
You can learn more about Neptune namespaces and fields in the [documentation](https://docs.neptune.ai/about/namespaces_and_fields/).

In [ ]:
run["model/performance/roc"].upload(mp)

### model_parts
This function calculates Variable Importance.

In [ ]:
vi = exp.model_parts().plot(show=False)
vi

There is also a possibility of calculating variable importance of group of variables

In [ ]:
vi_grouped = exp.model_parts(
    variable_groups={"personal": ["gender", "age", "sibsp", "parch"], "wealth": ["class", "fare"]}
).plot(show=False)
vi_grouped

#### (Neptune) Upload variable importance plots to Neptune

In [ ]:
run["model/variable_importance/single"].upload(vi)
run["model/variable_importance/grouped"].upload(vi_grouped)

### model_profile
This function calculates explanations that explore model response as a function of selected variables.  
The explanations can be calculated as Partial Dependence Profile or Accumulated Local Dependence Profile.

In [ ]:
pdp_num = exp.model_profile(type="partial", label="pdp")
ale_num = exp.model_profile(type="accumulated", label="ale")
mp_num = pdp_num.plot(ale_num, show=False)
mp_num

In [ ]:
pdp_cat = exp.model_profile(
    type="partial", variable_type="categorical", variables=["gender", "class"], label="pdp"
)
ale_cat = exp.model_profile(
    type="accumulated", variable_type="categorical", variables=["gender", "class"], label="ale"
)
mp_cat = ale_cat.plot(pdp_cat, show=False)
mp_cat

#### (Neptune) Upload model profile plots to Neptune

In [ ]:
run["model/profile/num"].upload(mp_num)
run["model/profile/cat"].upload(mp_cat)

## Prediction-level explanations

Let's create two example persons for this tutorial.

In [ ]:
john = pd.DataFrame(
    {
        "gender": ["male"],
        "age": [25],
        "class": ["1st"],
        "embarked": ["Southampton"],
        "fare": [72],
        "sibsp": [0],
        "parch": 0,
    },
    index=["John"],
)

mary = pd.DataFrame(
    {
        "gender": ["female"],
        "age": [35],
        "class": ["3rd"],
        "embarked": ["Cherbourg"],
        "fare": [25],
        "sibsp": [0],
        "parch": [0],
    },
    index=["Mary"],
)

### predict_parts
This function calculates Variable Attributions as Break Down, iBreakDown or Shapley Values explanations.  
Model prediction is decomposed into parts that are attributed for particular variables.

Breakdown values for John's predictions

In [ ]:
bd_john = exp.predict_parts(john, type="break_down", label=john.index[0])
bd_interactions_john = exp.predict_parts(john, type="break_down_interactions", label="John+")
bd_pp = bd_john.plot(bd_interactions_john, show=False)
bd_pp

Shapely values for Mary's predictions

In [ ]:
sh_mary = exp.predict_parts(mary, type="shap", B=10, label=mary.index[0]).plot(show=False)
sh_mary

#### (Neptune) Upload plots to Neptune

In [ ]:
run["prediction/breakdown/john"].upload(bd_pp)
run["prediction/shapely/mary"].upload(sh_mary)

### predict_profile

This function computes individual profiles aka Ceteris Paribus Profiles.

In [ ]:
cp_mary = exp.predict_profile(mary, label=mary.index[0])
cp_john = exp.predict_profile(john, label=john.index[0])
pred_profile = cp_mary.plot(cp_john, show=False)
pred_profile

In [ ]:
pred_profile_cat = cp_john.plot(cp_mary, variable_type="categorical", show=False)
pred_profile_cat

#### (Neptune) Upload CP plots to Neptune

In [ ]:
run["prediction/profile/numerical"].upload(pred_profile)
run["prediction/profile/categorical"].upload(pred_profile_cat)

## Stop logging

Once you are done logging, stop tracking the run.

In [ ]:
run.stop()

## Analyze reports in the Neptune app
Go to the run link and explore the reports. 
You can also explore this [example run](https://app.neptune.ai/o/common/org/dalex-support/runs/details?viewId=standard-view&detailsTab=dashboard&dashboardId=993ea4c1-c528-4d6d-86ba-1a7a3bd65e7e&shortId=DLX-2&type=run).